In [1]:
import os
import copy
import random
import logging
from pprint import pformat
from datetime import datetime
from contextlib import contextmanager
from pathlib import Path
from time import time
from math import cos, pi

import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from tqdm import tqdm
from PIL import Image
import torchvision
import sklearn.metrics
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler
from albumentations import CLAHE, HorizontalFlip, Compose, HueSaturationValue, RandomBrightness, RandomContrast, Normalize, Resize

In [2]:
class Config():
    def __init__(self, home=True):
        self.seed = 71
        self.batch_size = 16
        self.accum_time = 4
        self.train_dir = '../input/train'
        self.train_csv = '../input/train.csv'
        self.test_dir = '../input/test'
        self.test_csv = '../input/sample_submission.csv'
        self.device_name = 'cuda:0'
        self.image_size = 256
        self.n_splits = 5
        self.fold = 0
        self.num_epoch = 320
        self.lr_step_epoch = 64
        self.alpha = 1
        self.init_lr = 1e-3
        self.eta_min = 1e-6
        self.num_workers = 16 if home else 4
        self.classes_num = 5
    
conf = Config()

In [3]:
def now():
    return datetime.now().strftime("%Y_%m_%d_%H_%M_%S")


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)

    
def load_csv(path):
    return pd.read_csv(path)


def count_parameter(model):
    return sum(p.numel() for p in model.parameters())


def get_lr(optimizer):
    lr = list()
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    if len(lr) == 1:
        return lr[0]
    else:
        return lr

In [4]:
# setup for kernel
def setup(exp_name, config):
    """init experiment (directory setup etc...)"""

    result_dir = Path(f'../result/{exp_name}/')
    result_dir.mkdir(parents=True)

    set_seed(config.seed)

    device = torch.device(config.device_name)

    log = Logger(exp_name, result_dir / 'exp.log')

    log.info("configuration is following...")
    log.info(pformat(config.__dict__))

    return device, log, result_dir

In [5]:
class Logger:
    """Logging Uitlity Class for monitoring and debugging
    """

    def __init__(self,
                 name,
                 log_fname,
                 log_level=logging.INFO,
                 custom_log_handler=None):

        self.name = name
        self.logger = logging.getLogger(name)
        self.logger.setLevel(log_level)
        ch = logging.FileHandler(log_fname)
        self.logger.addHandler(ch)
        self.logger.addHandler(logging.StreamHandler())

        if custom_log_handler:
            if isinstance(custom_log_handler, list):
                for handler in custom_log_handler:
                    self.logger.addHandler(handler)
            else:
                self.logger.addHandler(handler)

    def kiritori(self):
        self.logger.info('-'*80)

    def double_kiritori(self):
        self.logger.info('='*80)

    def space(self):
        self.logger.info('\n')

    @contextmanager
    def interval_timer(self, name):
        start_time = datetime.now()
        self.logger.info("\n")
        self.logger.info(f"Execution {name} start at {start_time}")
        try:
            yield
        finally:
            end_time = datetime.now()
            td = end_time - start_time
            self.logger.info(f"Execution {name} end at {end_time}")
            self.logger.info(f"Execution Time : {td}")
            self.logger.info("\n")

    def __getattr__(self, attr):
        """
        for calling logging class attribute
        if you call attributes of other class, raise AttributeError
        """
        # self.logger.info(f"{datetime.now()}")
        return getattr(self.logger, attr)

In [6]:
valid_transform = Compose([
    Resize(conf.image_size, conf.image_size),
    Normalize(),
])

In [7]:
def convert_num(target):
    labels = np.zeros(conf.classes_num)
#     for t in range(target+1):
#         labels[t] = 1
    labels[target] = 1
    return labels.astype(np.float32)


class APTOSDataset(Dataset):
    def __init__(self,
                 root_dir,
                 data_csv,
                 augment=None,
                 test=False,
                mixup=False):
        super().__init__()
        self.root_dir = Path(root_dir)
        self.data_csv = data_csv
        self.augment = augment
        self.test = test
        self.mixup = mixup
        
    def do_mixup(self, img, label, alpha=1.):
        index = np.random.randint(0,len(self.data_csv))
        row = self.data_csv.loc[index]
        fname = f"{row.id_code}.png"
        fpath = self.root_dir / fname
        img2 = np.array(Image.open(fpath))
        if self.augment:
            img2 = self.augment(image=img2)['image']
            img2 = np.moveaxis(img2, -1, 0)
        
        label2 = row.diagnosis
        label2 = convert_num(label2)
        
        rate = np.random.beta(alpha,alpha)
        img = img*rate + img2*(1-rate)
        label = label*rate + label2*(1-rate)
        return img, label

    def __len__(self):
        return len(self.data_csv)

    def __getitem__(self, index):
        sample = dict()
        row = self.data_csv.loc[index]
        fname = f"{row.id_code}.png"
        fpath = self.root_dir / fname
        image = np.array(Image.open(fpath))
        
        if self.augment:
            image = self.augment(image=image)['image']
            image = np.moveaxis(image, -1, 0)
        
        if self.test != "test":
            label = convert_num(row.diagnosis)
            if self.mixup and np.random.random()<0.5:
                image, label = self.do_mixup(image, label)
            sample['label'] = label

        sample['data'] = np.array(image)

        return sample

In [8]:
def worker_init_fn(worker_id):                                                          
    np.random.seed(conf.seed + worker_id)

def make_loader(df,
                root_dir,
                batch_size=conf.batch_size,
                shuffle=True,
                test="train",
                image_dataset=False,
                worker_init_fn=worker_init_fn,
                **kwargs):

    ds = APTOSDataset(
        root_dir,
        df,
        test=test,
        **kwargs)

    drop_last = test == "test"
    loader = DataLoader(
        ds, batch_size=batch_size, shuffle=shuffle,
        num_workers=conf.num_workers,
        drop_last=drop_last)
    return loader, len(ds)

In [9]:
class ResNet(nn.Module):
    def __init__(self,
                 arch_name='resnet18',
                 input_channel=3,
                 input_size=224,
                 num_classes=28):
        super(ResNet, self).__init__()
        self.base_model = torchvision.models.__dict__[arch_name](pretrained="imagenet")
        if isinstance(input_size, tuple):
            ksize = (input_size[0] // 16, input_size[1] // 16)
        else:
            ksize = input_size // 16

        self.base_model.bn0 = nn.BatchNorm2d(input_channel)
        self.base_model.avgpool = nn.AvgPool2d(kernel_size=ksize)

        self.dim_feats = self.base_model.fc.in_features  # = 2048
        self.base_model.fc = nn.Linear(self.dim_feats, num_classes)
        self.out_size = ksize

    def forward(self, data):
        # x = self.base_model.bn0(data)
        x = self.base_model.conv1(data)
        x = self.base_model.bn1(x)
        x = self.base_model.relu(x)

        x = self.base_model.layer1(x)
        x = self.base_model.layer2(x)
        x = self.base_model.layer3(x)
        x = self.base_model.layer4(x)
        x = self.base_model.avgpool(x)
        x = x.view(-1, self.dim_feats)
        x = self.base_model.fc(x)

        return x

In [10]:
class CosineLR(_LRScheduler):
    """SGD with cosine annealing.
    """

    def __init__(self, optimizer, step_size_min=1e-5, t0=100, tmult=2, curr_epoch=-1, last_epoch=-1):
        self.step_size_min = step_size_min
        self.t0 = t0
        self.tmult = tmult
        self.epochs_since_restart = curr_epoch
        super(CosineLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        self.epochs_since_restart += 1

        if self.epochs_since_restart > self.t0:
            self.t0 *= self.tmult
            self.epochs_since_restart = 0

        lrs = [self.step_size_min + (
                    0.5 * (base_lr - self.step_size_min) * (1 + cos(self.epochs_since_restart * pi / self.t0)))
               for base_lr in self.base_lrs]

        # print(lrs)

        return lrs

In [11]:
# https://inclass.kaggle.com/gennadylaptev/qwk-loss-for-pytorch/data
# Categorical Crossentropyから途中で切り替えるのがいいらしい（https://arxiv.org/pdf/1612.00775.pdf）
def kappa_loss(p, y, n_classes=5, eps=1e-10):
    """
    QWK loss function as described in https://arxiv.org/pdf/1612.00775.pdf
    
    Arguments:
        p: a tensor with probability predictions, [batch_size, n_classes],
        y, a tensor with one-hot encoded class labels, [batch_size, n_classes]
    Returns:
        QWK loss
    """
    
    W = np.zeros((n_classes, n_classes))
    for i in range(n_classes):
        for j in range(n_classes):
            W[i,j] = (i-j)**2
    
    W = torch.from_numpy(W.astype(np.float32)).to(conf.device_name)
    
    p = p.sigmoid()
    O = torch.matmul(y.t(), p)
    E = torch.matmul(y.sum(dim=0).view(-1,1), p.sum(dim=0).view(1,-1)) / O.sum()
    
    return (W*O).sum() / ((W*E).sum() + eps)


def calc_loss(pred, labels, criterion):
    if isinstance(pred, list):
        pred_len = len(pred)
        pred_probs = 0
        clf_loss = 0
        for i, px in enumerate(pred):
            pred_probs += px.sigmoid().cpu().data.numpy()
            clf_loss += criterion(px, labels)
        return clf_loss / pred_len, pred_probs / pred_len
    else:
        pred_probs = pred.sigmoid().cpu().data.numpy()
        clf_loss = criterion(pred, labels)
        return clf_loss, pred_probs

In [12]:
def predict(model, test_df,
            aug,
            device,
            data_dir='input/train'):

    model.eval()

    dataloader, ds_size = make_loader(
        test_df,
        data_dir,
        conf.batch_size,
        shuffle=False,
        test="test",
        augment=aug)

    all_preds = []
    # Iterate over data.
    t = dataloader
    for i, samples in enumerate(t):
        with torch.set_grad_enabled(False):
            inputs = samples['data'].to(device)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().data.numpy())

    all_preds = np.concatenate(all_preds)
    return all_preds

In [14]:
def inference(test_df,
                model_path,
                criterion,
                log,
                device):

    # load best model weights
    model.load_state_dict(model_path)
    test_preds = predict(model, test_df, valid_transform,
                         device, data_dir=conf.test_dir)

    return test_preds

In [15]:
def main():
    exp_name = "2019_07_13_02_36_49"
    device, log, result_dir = setup(exp_name, conf)
    test_df = load_csv(conf.test_csv)
    fpath = "../result/2019_07_13_02_36_49/model_0.pkl"

    test_preds = inference(
        test_df,
        fpath,
        criterion,
        log,
        device)
    
    test_df['diagnosis'] = np.argmax(test_preds, axis=1)
    test_df.to_csv(f'../pred/{exp_name}.csv', index=False)

In [ ]:
main()

configuration is following...
{'accum_time': 4,
 'alpha': 1,
 'batch_size': 16,
 'classes_num': 5,
 'device_name': 'cuda:0',
 'eta_min': 1e-06,
 'fold': 0,
 'image_size': 256,
 'init_lr': 0.001,
 'lr_step_epoch': 64,
 'n_splits': 5,
 'num_epoch': 320,
 'num_workers': 16,
 'seed': 71,
 'test_csv': '../input/sample_submission.csv',
 'test_dir': '../input/test',
 'train_csv': '../input/train.csv',
 'train_dir': '../input/train'}
done
classification learning start
--------------------
parameters 23518283
Optimizer: Adam
Scheduler: CosineAnnealingLR, step_size=64
0	133.39	135.2	0.0010	0.3365	0.5650369490226512	0.3232	0.6703980569127402	
1	134.51	269.7	0.0010	0.3075	0.6305859955975965	0.2630	0.7635706249325117	
2	133.55	403.3	0.0010	0.2917	0.6410851067140959	0.2435	0.7405345682133839	
3	136.87	540.2	0.0010	0.2917	0.6578330352120282	0.2362	0.7894199312714777	
4	137.21	677.4	0.0010	0.2803	0.6745074637567295	0.2471	0.7467522441589554	
5	137.36	814.7	0.0010	0.2764	0.662803880670065	0.2333	0.7914